importing pandas library

In [ ]:
import pandas as pd
from google.colab import files
uploaded=files.upload()


In [ ]:
data=pd.read_csv('train.csv')
data.head()

Data exploration:shape of dataset,column names highlights,missing values,data types

In [ ]:
print("shape of dataset:",data.shape)
print("columns of dataset:",data.columns)
print(data.isnull().sum())
print(data.dtypes)
data.head()

  handling missing values

In [ ]:
print(data.isnull().sum())
threshold=len(data)*0.5
data=data.dropna(axis=1,thresh=threshold)
#handling missing numerical values with mean
nums_cols=data.select_dtypes(include=['int64','float64']).columns
data[nums_cols]=data[nums_cols].fillna(data[nums_cols].mean())
#categorical value with most frequent one
cat_cols=data.select_dtypes(include=['object']).columns
data[cat_cols]=data[cat_cols].fillna(data[cat_cols].mode().iloc[0])
print("\nchecking")
print(data.isnull().sum())

Here dropping columns which are not containing meaningful information

In [ ]:
irrelevant_cols=["index","beer/beerId","beer/brewerId","review/timeStruct","review/timeUnix","user/birthdayRaw","user/birthdayUnix","user/profileName"]
data=data.drop(irrelevant_cols,errors='ignore')
print("after drop")
print(data.columns)

PREPROCESSING TEXT AND CATEGORICAL FEATURES

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

tfidf=TfidfVectorizer(stop_words='english',max_features=500)
review_text_tfidf=tfidf.fit_transform(data['review/text'].fillna('')).toarray()

le_style=LabelEncoder()
data['beer/style']=le_style.fit_transform(data['beer/style'].fillna('Unknown'))
data['beer/name']=le_style.fit_transform(data['beer/name'].fillna('Unknown'))

import numpy as np
tfidf_features=pd.DataFrame(review_text_tfidf,columns=[f'tfidf_{i}' for i in range(review_text_tfidf.shape[1])])
data = pd.concat([data.reset_index(drop=True), tfidf_features.reset_index(drop=True)], axis=1)
data = data.drop(columns=['review/text'])

print("Data after preprocessing:")
print(data.head())


Splitting dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['review/overall'])
y = data['review/overall']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Training Features Shape:", X_train.shape)
print("Test Features Shape:", X_test.shape)
print("Training Labels Shape:", y_train.shape)
print("Test Labels Shape:", y_test.shape)

Training the model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train = X_train.drop(columns=['review/timeStruct', 'user/profileName'], errors='ignore')
X_test = X_test.drop(columns=['review/timeStruct', 'user/profileName'], errors='ignore')

rf_model = RandomForestRegressor(random_state=42, n_estimators=10)#you can use more estimators also
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")


testing of model

In [ ]:
new_data = X_test.iloc[10:11]
predicted_rating = rf_model.predict(new_data)

print(new_data)
print(f"Predicted Overall Rating: {predicted_rating[0]}")